In [1]:
import ee
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
from tqdm import tqdm

import subprocess
from joblib import Parallel, delayed
import multiprocessing

from sklearn.model_selection import StratifiedKFold, train_test_split, StratifiedGroupKFold
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold


import xgboost as xgb
import lightgbm as lgb
import catboost as catb

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)

In [4]:
path = './data/'

In [5]:
# Load files
train = pd.read_csv(f'{path}Train.csv')
test = pd.read_csv(f'{path}Test.csv')
sample_submission = pd.read_csv(f'{path}SampleSubmission.csv')

# Preview head of train
train.head()

,ID,Lat,Lon,Target
0,ID_SJ098E7S2SY9,34.162491,70.763668,0
1,ID_CWCD60FGJJYY,32.075695,48.492047,0
2,ID_R1XF70RMVGL3,14.542826,33.313483,1
3,ID_0ZBIDY0PEBVO,14.359480,33.284108,1
4,ID_C20R2C0AYIT0,14.419128,33.528450,0


In [6]:
lonlat_mapper = {
    '34-70': 'Afghanistan',
    '32-48': 'Iran',
    '14-33': 'Sudan',
}

In [7]:
train['location'] = train.apply(lambda x: f'{np.int8(x[1])}-{np.int8(x[2])}', axis=1).map(lonlat_mapper)
test['location'] = test.apply(lambda x: f'{np.int8(x[1])}-{np.int8(x[2])}', axis=1).map(lonlat_mapper)

In [8]:
train.head()

,ID,Lat,Lon,Target,location
0,ID_SJ098E7S2SY9,34.162491,70.763668,0,Afghanistan
1,ID_CWCD60FGJJYY,32.075695,48.492047,0,Iran
2,ID_R1XF70RMVGL3,14.542826,33.313483,1,Sudan
3,ID_0ZBIDY0PEBVO,14.359480,33.284108,1,Sudan
4,ID_C20R2C0AYIT0,14.419128,33.528450,0,Sudan


In [9]:
test.head()

,ID,Lat,Lon,location
0,ID_9ZLHTVF6NSU7,34.254835,70.348699,Afghanistan
1,ID_LNN7BFCVEZKA,32.009669,48.535526,Iran
2,ID_SOYSG7W04UH3,14.431884,33.399991,Sudan
3,ID_EAP7EXXV8ZDE,14.281866,33.441224,Sudan
4,ID_QPRX1TUQVGHU,14.399365,33.109566,Sudan


In [10]:
# Preview head of the sample submission
sample_submission.head()

,ID,Target
0,ID_9ZLHTVF6NSU7,
1,ID_LNN7BFCVEZKA,
2,ID_SOYSG7W04UH3,
3,ID_EAP7EXXV8ZDE,
4,ID_QPRX1TUQVGHU,


In [11]:
# Get authetication token and sign in to Google Earth Engine
ee.Authenticate()

Enter verification code:  4/1AfJohXmCt6LI_bA_6rsnLUfw4NCtfTx3VXcU14Lx-G9lfXpyUu_u6hIEPsg



Successfully saved authorization token.


In [12]:
ee.Initialize(project='geo-ai-cropland')

Load Sentinel-2 imagery from Earth Engine and select the df['bands.
In the example we use the median value, df['but other options might work as well.
It might df['be usefule to apply a cloud mask, to avoid odd values. Please see https://developers.google.com/earth-engine/tutorials/community/sentinel-2-s2cloudless for reference

Load the training dataset from CSV
(make sure the path fits with the location you stored the data) and transform in training points as Earth Engine features.

In [13]:
# Load the S2 image collection
s2_collection = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')

# Define the df['bands of interest
bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B11', 'B12', 'AOT', 'WVP', 'SCL', 'TCI_R', 'TCI_G', 'TCI_B']

In [14]:
CLOUD_FILTER = 10
time_maps = {
    'Afghanistan': {
        'start': '2021-04-01',
        'end': '2022-04-30',
    },
    'Iran': {
        'start': '2019-07-01',
        'end': '2020-06-30',
    },
    'Sudan': {
        'start': '2019-07-01',
        'end': '2020-06-30',
    }
}

def load_data(data):    
    all_values = []
    dfs = []
    for region in tqdm(time_maps.keys()):
        df = data[data.location == region]
        point_geometries = [ee.Geometry.Point(lon, lat) for lon, lat in zip(df['Lon'], df['Lat'])]

        collection = (
            s2_collection
            .filterDate(time_maps[region]['start'], time_maps[region]['end'])
            .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER))
        )

        mean_values = (
            collection
            .select(bands)
            .filterBounds(ee.FeatureCollection(point_geometries))
            .mean()
            .reduceRegions(collection=ee.FeatureCollection(point_geometries), reducer=ee.Reducer.mean(), scale=10, tileScale=2)
        )

        all_values.append(mean_values)
        dfs.append(df)

    results = []
    for mean_values in all_values:
        for feature in tqdm(mean_values.getInfo()['features']):
            values = [feature['properties'][band] for band in bands]
            results.append(values)

    final_df = pd.concat(dfs)
    final_df[bands] = results
    final_df = final_df.sort_index()


    return final_df

def load_ts_data(data):
    all_values = []
    dfs = []
    for region in tqdm(time_maps.keys()):
        df = data[data.location == region]
        point_geometries = [ee.Geometry.Point(lon, lat) for lon, lat in zip(df['Lon'], df['Lat'])]

        collection = (
            s2_collection
            .filterDate(time_maps[region]['start'], time_maps[region]['end'])
            .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER))
        )

        mean_values = (
            collection
            .filterBounds(ee.FeatureCollection(point_geometries))
            .select(bands)
            .toBands()
            .reduceRegions(collection=ee.FeatureCollection(point_geometries), reducer=ee.Reducer.mean(), scale=10, tileScale=1)
        )

        all_values.append(mean_values)
        dfs.append(df)

    results = {k:[] for k in ['ID'] + bands}
    for df, mean_values in zip(dfs, all_values):
        for idx, feature in tqdm(zip(df.ID.values, mean_values.getInfo()['features'])):
            length = len(feature['properties'].keys()) // len(bands)

            results['ID'].extend([idx] * length)
            for k, v in feature['properties'].items():
                band = '_'.join(k.split('_')[3:])
                results[band].append(v)

    final_df = pd.DataFrame(results).dropna()

    return final_df

In [15]:
%%time
train_data = load_data(train)
test_data = load_data(test)

100%|██████████| 500/500 [00:00<00:00, 492983.54it/s]

CPU times: user 517 ms, sys: 3.7 ms, total: 521 ms
Wall time: 4min 21s


In [16]:
%%time
train_ts_data = load_ts_data(train)
test_ts_data = load_ts_data(test)

100%|██████████| 3/3 [00:00<00:00, 86.48it/s]
500it [00:00, 2540.77it/s]
500it [00:00, 1068.97it/s]
500it [00:00, 2144.18it/s]
100%|██████████| 3/3 [00:00<00:00, 81.59it/s]
500it [00:00, 2507.19it/s]
500it [00:00, 961.09it/s]
500it [00:00, 2156.57it/s]


CPU times: user 5.4 s, sys: 409 ms, total: 5.81 s
Wall time: 1min 38s


In [18]:
train_data.to_csv('data/train_data.csv', index=False)
test_data.to_csv('data/test_data.csv', index=False)

train_ts_data.to_csv('data/train_ts_data.csv', index=False)
test_ts_data.to_csv('data/test_ts_data.csv', index=False)